In [2]:
!pip install streamlit pyngrok ultralytics opencv-python-headless
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip install cython_bbox

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 108.9 MB/s eta 0:00:00
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-76r600km
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-76r600km
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Preparing metadata (setup.py) ... done
  Created wheel for pycocotools: filename=pycocotools-2.0-cp312-cp312-linux_x86_64.whl size=426680 sha256=a096474672408db74aa87cb2bba69828a1e83ed114acd731b400baf6d572bc99
  Stored in directory: /tmp/pip-ephem-wheel-cache-6nmidlsn/wheels/95/e6/c7/8ceda667bca7218619fea052622a0b11a37fb51c28c993fae3
Successfully built pycocotools
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2

In [12]:
%%writefile app.py
import streamlit as st
import os
import tempfile
import json

try:
    from video_tracker import track_video
    TRACKER_AVAILABLE = True
except ImportError as e:
    st.error(f"Error importing video_tracker: {e}")
    TRACKER_AVAILABLE = False

st.set_page_config(
    page_title="Vehicle & Pedestrian Tracker",
    page_icon="🚦",
    layout="wide"
)

st.title("🚦 Vehicle and Pedestrian Tracking with YOLOv8 & ByteTrack")
st.markdown("Upload a video to track vehicles and pedestrians using YOLOv8 with ByteTrack tracking")

MODEL_WEIGHTS_PATH = "best.pt"

if not TRACKER_AVAILABLE:
    st.error("Tracking functionality not available. Please check video_tracker.py file.")
elif not os.path.exists(MODEL_WEIGHTS_PATH):
    st.error(f"❌ Model weights file not found at '{MODEL_WEIGHTS_PATH}'")
else:
    st.success(f"✅ Model found at: {MODEL_WEIGHTS_PATH}")

    uploaded_file = st.file_uploader("Upload a video file", type=["mp4", "mov", "avi", "mkv"])

    if uploaded_file is not None:
        file_size = uploaded_file.size / (1024 * 1024)
        st.info(f"Uploaded video: {uploaded_file.name} ({file_size:.2f} MB)")

        with tempfile.NamedTemporaryFile(delete=False, suffix='.mp4') as tfile:
            tfile.write(uploaded_file.getbuffer())
            input_video_path = tfile.name

        st.subheader("Original Video")
        st.video(input_video_path)

        if st.button("🎯 Start Tracking"):

            output_video_path = f"tracked_{uploaded_file.name}"
            results_json_path = "tracking_results.json"

            progress_bar = st.progress(0.0)
            status_text = st.empty()

            def update_progress(frame_idx, total_frames):
                progress = frame_idx / total_frames
                progress_bar.progress(progress)
                status_text.text(f"Processing frame {frame_idx}/{total_frames}")

            with st.spinner("Processing video... This may take a few minutes depending on video length."):
                success, message = track_video(
                    input_video_path,
                    output_video_path,
                    MODEL_WEIGHTS_PATH,
                    results_json_path,
                    progress_callback=update_progress
                )

            if success:
                progress_bar.progress(1.0)
                status_text.success("✅ Processing Complete!")
                st.success("🎉 Tracking completed successfully!")

                col1, col2 = st.columns(2)

                with col1:
                    st.subheader("Tracked Video")
                    if os.path.exists(output_video_path):
                        st.video(output_video_path)
                    else:
                        st.error("Output video file not found")

                with col2:
                    st.subheader("Tracking Results")
                    if os.path.exists(results_json_path):
                        with open(results_json_path, 'r') as f:
                            results_data = json.load(f)

                        tracking_results = results_data.get('tracking_results', [])
                        total_frames = len(tracking_results)

                        all_objects = []
                        for frame in tracking_results:
                            all_objects.extend(frame.get('objects', []))

                        if all_objects:
                            unique_objects = len(set(obj['id'] for obj in all_objects))
                            total_detections = len(all_objects)

                            class_counts = {}
                            for obj in all_objects:
                                cls = obj['class']
                                class_counts[cls] = class_counts.get(cls, 0) + 1

                            st.metric("Total Frames Processed", total_frames)
                            st.metric("Unique Objects Tracked", unique_objects)
                            st.metric("Total Detections", total_detections)

                            st.subheader("Object Distribution")
                            for cls, count in class_counts.items():
                                st.metric(f"{cls.title()}", count)
                        else:
                            st.metric("Total Frames Processed", total_frames)
                            st.info("No objects detected in the video")

                st.markdown("---")
                st.subheader("📥 Download Results")

                col1, col2 = st.columns(2)
                with col1:
                    if os.path.exists(results_json_path):
                        with open(results_json_path, "rb") as f:
                            st.download_button(
                                label="Download Tracking Results (JSON)",
                                data=f,
                                file_name="tracking_results.json",
                                mime="application/json"
                            )
                with col2:
                    if os.path.exists(output_video_path):
                        with open(output_video_path, "rb") as f:
                            st.download_button(
                                label="Download Tracked Video",
                                data=f,
                                file_name=output_video_path,
                                mime="video/mp4"
                            )

            else:
                st.error(f"❌ Processing failed: {message}")

            # Clean up temp input video
            try:
                os.unlink(input_video_path)
            except:
                pass

st.markdown("---")
st.subheader("📋 Instructions")
st.markdown("""
1. Upload Video (MP4, MOV, AVI, MKV)
2. Click 'Start Tracking'
3. Wait while frames are processed
4. Watch tracked video and view statistics
5. Download results (JSON or video)
""")

with st.sidebar.expander("Model Information"):
    if os.path.exists(MODEL_WEIGHTS_PATH):
        st.write(f"*Model*: {MODEL_WEIGHTS_PATH}")
        st.write("*Tracker*: ByteTrack")
        st.write("*Classes*: Vehicles and Pedestrians")

with st.sidebar.expander("Debug Info"):
    st.write(f"Current directory: {os.getcwd()}")
    st.write("Files in directory:")
    for file in os.listdir('.'):
        st.write(f"- {file}")


Overwriting app.py


In [16]:
%%writefile video_tracker.py
import cv2
import json
from ultralytics import YOLO
import numpy as np

def track_video(input_path, output_path, model_path, results_json_path, progress_callback=None):
    """
    Track objects in video using YOLOv8 and return tracking results
    progress_callback: optional function(frame_idx, total_frames) to report progress
    """
    try:
        # Load YOLOv8 model
        model = YOLO(model_path)

        # Open video
        cap = cv2.VideoCapture(input_path)

        # Get video properties
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Initialize video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        tracking_results = []
        frame_count = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Track objects on the frame
            results = model.track(frame, persist=True, verbose=False)

            frame_data = {'frame': frame_count, 'objects': []}

            # Process results
            if results[0].boxes is not None and results[0].boxes.id is not None:
                boxes = results[0].boxes.xyxy.cpu().numpy()
                track_ids = results[0].boxes.id.cpu().numpy()
                confidences = results[0].boxes.conf.cpu().numpy()
                classes = results[0].boxes.cls.cpu().numpy()

                for box, track_id, conf, cls in zip(boxes, track_ids, confidences, classes):
                    x1, y1, x2, y2 = box
                    obj_data = {
                        'id': int(track_id),
                        'class': model.names[int(cls)],
                        'confidence': float(conf),
                        'bbox': [float(x1), float(y1), float(x2), float(y2)]
                    }
                    frame_data['objects'].append(obj_data)

                    # Draw bounding box and ID on frame
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                    cv2.putText(frame, f'ID: {int(track_id)} {model.names[int(cls)]}',
                                (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            tracking_results.append(frame_data)
            out.write(frame)

            frame_count += 1
            if progress_callback:
                progress_callback(frame_count, total_frames)  # Report progress

        # Cleanup
        cap.release()
        out.release()

        # Save tracking results
        results_data = {
            'video_info': {'total_frames': total_frames, 'fps': fps, 'width': width, 'height': height},
            'tracking_results': tracking_results
        }
        with open(results_json_path, 'w') as f:
            json.dump(results_data, f, indent=2)

        return True, "Tracking completed successfully"

    except Exception as e:
        return False, str(e)


Overwriting video_tracker.py


In [17]:
# Download YOLOv8 model weights (you can use a pre-trained or custom model)
from ultralytics import YOLO

# Download a pre-trained YOLOv8 model
model = YOLO('yolov8n.pt')  # or yolov8s.pt, yolov8m.pt, yolov8l.pt, yolov8x.pt

# Save as best.pt (as expected by the app)
model.save('best.pt')
print("Model downloaded and saved as 'best.pt'")

Model downloaded and saved as 'best.pt'


In [18]:
import subprocess
import threading
import time
from pyngrok import ngrok

# Set your ngrok auth token (get it from https://dashboard.ngrok.com/get-started/your-authtoken)
ngrok.set_auth_token("339RnrCDGfIPajXXgQgpfY1CZ0W_2gG21oMUgSo3xXipkeWPV")  # Replace with your actual token

# Function to run Streamlit
def run_streamlit():
    subprocess.run(["streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"])

# Start Streamlit in background thread
thread = threading.Thread(target=run_streamlit)
thread.daemon = True
thread.start()

# Wait for Streamlit to start
time.sleep(10)

# Create ngrok tunnel
public_url = ngrok.connect(8501)
print(f"🚀 Your Streamlit app is running at: {public_url}")
print("Click the link above to access your application!")

🚀 Your Streamlit app is running at: NgrokTunnel: "https://loessial-cosmonautically-verda.ngrok-free.dev" -> "http://localhost:8501"
Click the link above to access your application!


In [8]:
# -------------------------------
# 1️⃣ Install dependencies
# -------------------------------
!pip install streamlit pyngrok ultralytics opencv-python-headless
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

# -------------------------------
# 2️⃣ Download YOLOv8 model
# -------------------------------
from ultralytics import YOLO

model = YOLO('yolov8n.pt')  # pre-trained nano model
model.save('best.pt')        # save as best.pt for app
print("✅ YOLOv8 model downloaded as 'best.pt'")

# -------------------------------
# 3️⃣ Create video_tracker.py
# -------------------------------
video_tracker_code = """
import cv2
import json
from ultralytics import YOLO
import numpy as np

def track_video(input_path, output_path, model_path, results_json_path):
    try:
        model = YOLO(model_path)
        cap = cv2.VideoCapture(input_path)
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        tracking_results = []
        frame_count = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            results = model.track(frame, persist=True, verbose=False)
            frame_data = {'frame': frame_count, 'objects': []}

            if results[0].boxes is not None and results[0].boxes.id is not None:
                boxes = results[0].boxes.xyxy.cpu().numpy()
                track_ids = results[0].boxes.id.cpu().numpy()
                confidences = results[0].boxes.conf.cpu().numpy()
                classes = results[0].boxes.cls.cpu().numpy()

                for box, track_id, conf, cls in zip(boxes, track_ids, confidences, classes):
                    x1, y1, x2, y2 = box
                    obj_data = {
                        'id': int(track_id),
                        'class': model.names[int(cls)],
                        'confidence': float(conf),
                        'bbox': [float(x1), float(y1), float(x2), float(y2)]
                    }
                    frame_data['objects'].append(obj_data)
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,255,0), 2)
                    cv2.putText(frame, f'ID: {int(track_id)} {model.names[int(cls)]}',
                                (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

            tracking_results.append(frame_data)
            out.write(frame)
            frame_count += 1

        cap.release()
        out.release()

        results_data = {'video_info': {'total_frames': total_frames,'fps': fps,'width': width,'height': height},
                        'tracking_results': tracking_results}

        with open(results_json_path, 'w') as f:
            json.dump(results_data, f, indent=2)

        return True, "Tracking completed successfully"
    except Exception as e:
        return False, str(e)
"""

with open("video_tracker.py", "w") as f:
    f.write(video_tracker_code)

# -------------------------------
# 4️⃣ Create app.py
# -------------------------------
app_code = """
import streamlit as st
import os, tempfile, json
from video_tracker import track_video

st.set_page_config(page_title="Vehicle & Pedestrian Tracker", page_icon="🚦", layout="wide")
st.title("🚦 Vehicle & Pedestrian Tracking with YOLOv8 & ByteTrack")
st.markdown("Upload a video to track vehicles and pedestrians")

MODEL_WEIGHTS_PATH = "best.pt"

if not os.path.exists(MODEL_WEIGHTS_PATH):
    st.error(f"❌ Model weights file not found at '{MODEL_WEIGHTS_PATH}'")
else:
    uploaded_file = st.file_uploader("Upload a video file", type=["mp4","mov","avi","mkv"])
    if uploaded_file is not None:
        with tempfile.NamedTemporaryFile(delete=False, suffix='.mp4') as tfile:
            tfile.write(uploaded_file.getbuffer())
            input_video_path = tfile.name

        st.video(input_video_path)
        if st.button("🎯 Start Tracking"):
            output_video_path = f"tracked_{uploaded_file.name}"
            results_json_path = "tracking_results.json"
            success, message = track_video(input_video_path, output_video_path, MODEL_WEIGHTS_PATH, results_json_path)
            if success:
                st.success("✅ Tracking completed!")
                st.video(output_video_path)
                with open(results_json_path, "r") as f:
                    results = json.load(f)
                st.json(results)
            else:
                st.error(f"❌ Processing failed: {message}")
"""

with open("app.py", "w") as f:
    f.write(app_code)

# -------------------------------
# 5️⃣ Run Streamlit + ngrok for public URL
# -------------------------------
from pyngrok import ngrok
import subprocess
import threading
import time

# Set your ngrok auth token here
NGROK_AUTH_TOKEN = "339RnrCDGfIPajXXgQgpfY1CZ0W_2gG21oMUgSo3xXipkeWPV"  # Replace this with your token from https://dashboard.ngrok.com/get-started/your-authtoken
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

def run_streamlit():
    subprocess.run(["streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"])

# Run Streamlit in a background thread
thread = threading.Thread(target=run_streamlit)
thread.start()

# Wait for Streamlit to start
time.sleep(10)

# Open public URL
public_url = ngrok.connect(8501)
print(f"🚀 Your public Streamlit URL: {public_url}")


  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-oqcte378
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-oqcte378
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Preparing metadata (setup.py) ... done
✅ YOLOv8 model downloaded as 'best.pt'
🚀 Your public Streamlit URL: NgrokTunnel: "https://loessial-cosmonautically-verda.ngrok-free.dev" -> "http://localhost:8501"


In [10]:
# -------------------------------
# 1️⃣ Install dependencies
# -------------------------------
!pip install streamlit pyngrok ultralytics opencv-python-headless
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

# -------------------------------
# 2️⃣ Download YOLOv8 model
# -------------------------------
from ultralytics import YOLO

# Download YOLOv8n pre-trained model and save as best.pt
model = YOLO('yolov8n.pt')
model.save('best.pt')
print("✅ YOLOv8 model downloaded as 'best.pt'")

# -------------------------------
# 3️⃣ Create video_tracker.py
# -------------------------------
video_tracker_code = """
import cv2
import json
from ultralytics import YOLO

def track_video(input_path, output_path, model_path, results_json_path):
    try:
        model = YOLO(model_path)
        cap = cv2.VideoCapture(input_path)
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        tracking_results = []
        frame_count = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            results = model.track(frame, persist=True, verbose=False)
            frame_data = {'frame': frame_count, 'objects': []}

            if results[0].boxes is not None and results[0].boxes.id is not None:
                boxes = results[0].boxes.xyxy.cpu().numpy()
                track_ids = results[0].boxes.id.cpu().numpy()
                confidences = results[0].boxes.conf.cpu().numpy()
                classes = results[0].boxes.cls.cpu().numpy()

                for box, track_id, conf, cls in zip(boxes, track_ids, confidences, classes):
                    x1, y1, x2, y2 = box
                    obj_data = {
                        'id': int(track_id),
                        'class': model.names[int(cls)],
                        'confidence': float(conf),
                        'bbox': [float(x1), float(y1), float(x2), float(y2)]
                    }
                    frame_data['objects'].append(obj_data)
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,255,0), 2)
                    cv2.putText(frame, f'ID: {int(track_id)} {model.names[int(cls)]}',
                                (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

            tracking_results.append(frame_data)
            out.write(frame)
            frame_count += 1

        cap.release()
        out.release()

        results_data = {'video_info': {'total_frames': total_frames,'fps': fps,'width': width,'height': height},
                        'tracking_results': tracking_results}

        with open(results_json_path, 'w') as f:
            json.dump(results_data, f, indent=2)

        return True, "Tracking completed successfully"
    except Exception as e:
        return False, str(e)
"""

with open("video_tracker.py", "w") as f:
    f.write(video_tracker_code)

# -------------------------------
# 4️⃣ Create app.py
# -------------------------------
app_code = """
import streamlit as st
import os, tempfile, json
from video_tracker import track_video

st.set_page_config(page_title="Vehicle & Pedestrian Tracker", page_icon="🚦", layout="wide")
st.title("🚦 Vehicle & Pedestrian Tracking with YOLOv8 & ByteTrack")
st.markdown("Upload a video to track vehicles and pedestrians")

MODEL_WEIGHTS_PATH = "best.pt"

if not os.path.exists(MODEL_WEIGHTS_PATH):
    st.error(f"❌ Model weights file not found at '{MODEL_WEIGHTS_PATH}'")
else:
    uploaded_file = st.file_uploader("Upload a video file", type=["mp4","mov","avi","mkv"])
    if uploaded_file is not None:
        with tempfile.NamedTemporaryFile(delete=False, suffix='.mp4') as tfile:
            tfile.write(uploaded_file.getbuffer())
            input_video_path = tfile.name

        st.subheader("Original Video")
        st.video(input_video_path)

        if st.button("🎯 Start Tracking"):
            output_video_path = f"tracked_{uploaded_file.name}"
            results_json_path = "tracking_results.json"

            success, message = track_video(input_video_path, output_video_path, MODEL_WEIGHTS_PATH, results_json_path)

            if success:
                st.success("✅ Tracking completed successfully!")

                col1, col2 = st.columns(2)

                with col1:
                    st.subheader("Tracked Video")
                    st.video(output_video_path)

                with col2:
                    st.subheader("Tracking Results")
                    with open(results_json_path, "r") as f:
                        results_data = json.load(f)

                    tracking_results = results_data.get("tracking_results", [])
                    total_frames = len(tracking_results)

                    all_objects = []
                    for frame in tracking_results:
                        all_objects.extend(frame.get("objects", []))

                    unique_objects = len(set(obj["id"] for obj in all_objects))
                    total_detections = len(all_objects)

                    class_counts = {}
                    for obj in all_objects:
                        cls = obj["class"]
                        class_counts[cls] = class_counts.get(cls, 0) + 1

                    st.metric("Total Frames Processed", total_frames)
                    st.metric("Unique Objects Tracked", unique_objects)
                    st.metric("Total Detections", total_detections)

                    st.subheader("Object Distribution")
                    for cls, count in class_counts.items():
                        st.metric(f"{cls.title()}", count)

                    st.markdown("---")
                    st.subheader("📥 Download Results")
                    col1, col2 = st.columns(2)
                    with col1:
                        with open(results_json_path, "rb") as f:
                            st.download_button("Download JSON", f, file_name="tracking_results.json")
                    with col2:
                        with open(output_video_path, "rb") as f:
                            st.download_button("Download Video", f, file_name=output_video_path)

            else:
                st.error(f"❌ Processing failed: {message}")
"""

with open("app.py", "w") as f:
    f.write(app_code)

# -------------------------------
# 5️⃣ Run Streamlit + ngrok for public URL
# -------------------------------
from pyngrok import ngrok
import subprocess, threading, time

# Replace with your ngrok auth token
NGROK_AUTH_TOKEN = "339RnrCDGfIPajXXgQgpfY1CZ0W_2gG21oMUgSo3xXipkeWPV"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

def run_streamlit():
    subprocess.run(["streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"])

# Run Streamlit in background thread
thread = threading.Thread(target=run_streamlit)
thread.start()

# Wait for Streamlit to start
time.sleep(10)

public_url = ngrok.connect(8501)
print(f"🚀 Your public Streamlit URL: {public_url}")


  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-6rawoca0
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-6rawoca0
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Preparing metadata (setup.py) ... done
✅ YOLOv8 model downloaded as 'best.pt'
🚀 Your public Streamlit URL: NgrokTunnel: "https://loessial-cosmonautically-verda.ngrok-free.dev" -> "http://localhost:8501"
